In [11]:
import numpy as np
from sklearn import preprocessing
raw_csv_data = np.loadtxt('creditcard1.csv', delimiter=',')
unscaled_inputs_all = raw_csv_data[:,0:-1] 
targets_all = raw_csv_data[:,-1]

In [12]:
# count the number of targets that contain 1
num_one_targets = int(np.sum(targets_all))
zero_targets_counter = 0
# remove the index from the tuple
indices_to_remove = []

# loop through all the 
for i in range(targets_all.shape[0]):
    # increase the zero counter by 1 if a target cell is 0
    if targets_all[i] == 0:
        zero_targets_counter += 1
        #if the target ith position is 0 and there are more zeros than 1s, remove the i index
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)

# assign new instance variables of the balanced dataset
unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)

In [13]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

In [14]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)
shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

In [15]:
samples_count = shuffled_inputs.shape[0]

# Count the samples in each subset, assuming we want 80-10-10 distribution of training, validation, and test.
# Naturally, the numbers are integers.
train_samples_count = int(0.8 * samples_count)
validation_samples_count = int(0.1 * samples_count)

# The 'test' dataset contains all remaining data.
test_samples_count = samples_count - train_samples_count - validation_samples_count

# Create variables that record the inputs and targets for training
# In our shuffled dataset, they are the first "train_samples_count" observations
train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

# Create variables that record the inputs and targets for validation.
# They are the next "validation_samples_count" observations, folllowing the "train_samples_count" we already assigned
validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

# Create variables that record the inputs and targets for test.
# They are everything that is remaining.
test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

# We balanced our dataset to be 50-50 (for targets 0 and 1), but the training, validation, and test were 
# taken from a shuffled dataset. Check if they are balanced, too. Note that each time you rerun this code, 
# you will get different values, as each time they are shuffled randomly.
# Normally you preprocess ONCE, so you need not rerun this code once it is done.
# If you rerun this whole sheet, the npzs will be overwritten with your newly preprocessed data.

# Print the number of targets that are 1s, the total number of samples, and the proportion for training, validation, and test.
print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

400.0 787 0.5082592121982211
47.0 98 0.47959183673469385
45.0 99 0.45454545454545453


In [16]:
np.savez('creditcard_fraud_train', inputs=train_inputs, targets=train_targets)
np.savez('creditcard_fraud_test', inputs=test_inputs, targets=test_targets)
np.savez('creditcard_fraud_validation', inputs=validation_inputs, targets=validation_targets)